In [1]:
staticDF = spark.read.format('csv')\
                .option('header','true')\
                .option('inferSchema','true')\
                .load('data/retail-data/by-day/*.csv')

In [2]:
staticDF.createOrReplaceTempView('retail_data')
staticSchema = staticDF.schema
staticSchema

StructType([StructField('InvoiceNo', StringType(), True), StructField('StockCode', StringType(), True), StructField('Description', StringType(), True), StructField('Quantity', IntegerType(), True), StructField('InvoiceDate', TimestampType(), True), StructField('UnitPrice', DoubleType(), True), StructField('CustomerID', DoubleType(), True), StructField('Country', StringType(), True)])

In [3]:
from pyspark.sql.functions import window, column, desc, col
staticDF.selectExpr("CustomerId", "(UnitPrice * Quantity) as total_cost", "InvoiceDate")\
        .groupBy(col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
        .sum("total_cost")\
        .show(5)

+----------+--------------------+-----------------+
|CustomerId|              window|  sum(total_cost)|
+----------+--------------------+-----------------+
|   16057.0|{2011-12-04 19:00...|            -37.6|
|   14126.0|{2011-11-28 19:00...|643.6300000000001|
|   13500.0|{2011-11-15 19:00...|497.9700000000001|
|   17160.0|{2011-11-07 19:00...|516.8499999999999|
|   15608.0|{2011-11-10 19:00...|            122.4|
+----------+--------------------+-----------------+
only showing top 5 rows



In [4]:
streamingDF = spark.readStream.schema(staticSchema)\
                   .option("maxFilesPerTrigger", 1)\
                   .format("csv")\
                   .option("header","true")\
                   .load("data/retail-data/by-day/*.csv")

In [5]:
streamingDF.isStreaming

True

In [6]:
purchaseByCustomerPerHour = streamingDF.selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate"
).groupBy(col("CustomerId"), window(col("InvoiceDate"), "1 day")).sum("total_cost")

In [7]:
purchaseByCustomerPerHour.writeStream.format("memory")\
    .queryName("customer_purchases")\
    .outputMode("complete")\
    .start()

In [15]:
spark.sql("""
SELECT *
FROM customer_purchases
ORDER BY `sum(total_cost)` DESC
""").show(5)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|      null|{2011-11-13 19:00...|          55316.08|
|      null|{2011-11-06 19:00...|          42939.17|
|      null|{2011-03-28 20:00...| 33521.39999999998|
|      null|{2011-12-07 19:00...|31975.590000000007|
|   18102.0|{2011-09-14 20:00...|31661.540000000005|
+----------+--------------------+------------------+
only showing top 5 rows



In [16]:
staticDF.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [17]:
from pyspark.sql.functions import date_format, col
preppedDF = staticDF.na.fill(0)\
    .withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE"))\
    .coalesce(5)

In [18]:
trainDF = preppedDF.where("InvoiceDate < '2011-07-01'")
testDF = preppedDF.where("InvoiceDate >= '2011-07-01'")

In [20]:
trainDF.count()

245903

In [21]:
testDF.count()

296006

In [23]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()\
    .setInputCol("day_of_week")\
    .setOutputCol("day_of_week_index")

In [25]:
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder()\
    .setInputCol("day_of_week_index")\
    .setOutputCol("day_of_week_encoded")

In [26]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler()\
    .setInputCols(["UnitPrice","Quantity","day_of_week_encoded"])\
    .setOutputCol("features")

In [27]:
from pyspark.ml import Pipeline
transformationPipeline = Pipeline()\
    .setStages([indexer, encoder, vectorAssembler])

In [28]:
fittedPipeline = transformationPipeline.fit(trainDF)

In [29]:
transformedTraining = fittedPipeline.transform(trainDF)

In [31]:
from pyspark.ml.clustering import KMeans
kmeans = KMeans()\
    .setK(20)\
    .setSeed(1)

In [32]:
kmodel = kmeans.fit(transformedTraining)

In [35]:
transformedTest = fittedPipeline.transform(testDF)